In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras_preprocessing.image import load_img
from keras_preprocessing.image import ImageDataGenerator
from keras.applications.xception import Xception, preprocess_input, decode_predictions

In [2]:
path ='./clothing-dataset-small/train/t-shirt/'
file = '1cab0eb2-60f6-4d0a-a380-9a07b834d855.jpg'
fullName = f'{path}/{file}'

In [ ]:
img = load_img(fullName, target_size=(299,299))

print(img)

x = np.array(img)
x.shape

In [ ]:
model = Xception(weights='imagenet', input_shape=(299,299,3))

X = np.array([x])
X = preprocess_input(X)

pred = model.predict(X)

decode_predictions(pred)

### CNN - Transfer Learning

In [ ]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_ds = train_gen.flow_from_directory('./clothing-dataset-small/train/',
                                         target_size=(150,150),                              
                                         batch_size=32)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = val_gen.flow_from_directory('./clothing-dataset-small/validation/',
                                     target_size=(150,150),
                                     batch_size=32,
                                     shuffle=False
                                    )

In [ ]:
print(train_ds.class_indices)

X, y = next(train_ds)

print(y[:5])

In [ ]:
base_model = tf.keras.applications.Xception(weights='imagenet',
                                            include_top=False,
                                            input_shape=(150, 150, 3))

base_model.trainable = False

inputs = keras.Input(shape=(150,150,3))

base = base_model(inputs, training=False)

vectors = keras.layers.GlobalAveragePooling2D()(base)

outputs = keras.layers.Dense(10)(vectors)

model = keras.Model(inputs, outputs)

In [ ]:
preds = model.predict(X)

In [ ]:
preds.shape

In [ ]:
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
loss = keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
%%time
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

In [ ]:
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.xticks(np.arange(10))
plt.legend()
plt.show()

In [ ]:
history.history

### Adjusting the learning rate

In [ ]:
def make_model(learning_rate = 0.01):
    
    base_model = tf.keras.applications.Xception(weights='imagenet',
                                            include_top=False,
                                            input_shape=(150, 150, 3))

    base_model.trainable = False

    #########################################################

    inputs = keras.Input(shape=(150,150,3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    outputs = keras.layers.Dense(10)(vectors)
    model = keras.Model(inputs, outputs)
    
    #########################################################
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
    return model

In [ ]:
%%time
scores = {}
lrs = [0.0001, 0.001, 0.01, 0.1]

for lr in lrs:
    print(lr)
    model = make_model(learning_rate=lr)
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[lr] = history.history
    print('\n\n')

In [ ]:
for lr , hist in scores.items():
    plt.plot(hist['val_accuracy'], label=lr)
    
plt.xticks(np.arange(10))    
plt.legend()

In [ ]:
model.save_weights('model_v1.h5', save_format='h5')

### Checkpointintg

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint('xception_v1_{epoch:02d}_{val_accuracy:.3f}.h5',
                                             save_best_only=True,
                                             monitor='val_accuracy',
                                             mode='max')

In [ ]:
lr = 0.01
print(lr)
model = make_model(learning_rate=lr)
history = model.fit(train_ds, epochs=10, validation_data=val_ds, callbacks=checkpoint)

In [ ]:
def make_modelv2(learning_rate = 0.001, size_inner=100):
    
    base_model = tf.keras.applications.Xception(weights='imagenet',
                                            include_top=False,
                                            input_shape=(150, 150, 3))

    base_model.trainable = False

    #########################################################

    inputs = keras.Input(shape=(150,150,3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    
    outputs = keras.layers.Dense(10)(inner)
    model = keras.Model(inputs, outputs)
    
    #########################################################
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
    return model

In [ ]:
tf.debugging.set_log_device_placement(True)

scores = {}
lr = 0.001

#with tf.device('/GPU:0'):

for size in [16, 32]:
    print(size)
    model = make_modelv2(learning_rate=lr, size_inner=size)
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[size] = history.history
    print('\n\n')

In [ ]:
for size , hist in scores.items():
    plt.plot(hist['val_accuracy'], label=('val=%s' % size))
    
plt.xticks(np.arange(10))    
_ = plt.legend()

In [ ]:
def make_modelv3(learning_rate = 0.001, size_inner=100, droprate=0.0):
    
    base_model = tf.keras.applications.Xception(weights='imagenet',
                                            include_top=False,
                                            input_shape=(150, 150, 3))

    base_model.trainable = False

    #########################################################

    inputs = keras.Input(shape=(150,150,3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = keras.layers.Dropout(droprate)(inner)
    
    outputs = keras.layers.Dense(10)(drop)
    model = keras.Model(inputs, outputs)
    
    #########################################################
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
    return model

In [ ]:
scores = {}
lr = 0.001
size = 32

for droprate in [0.0, 0.2, 0.4]:
    print(droprate)
    model = make_modelv2(learning_rate=lr, size_inner=size)
    history = model.fit(train_ds, epochs=20, validation_data=val_ds)
    scores[droprate] = history.history
    print('\n\n')

In [ ]:
for droprate , hist in scores.items():
    plt.plot(hist['val_accuracy'], label=('DropRate=%s' % droprate))
    
plt.xticks(np.arange(20))    
_ = plt.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

ax = plt.plot(scores[0.2]['val_accuracy'], label='Val')
ax = plt.plot(scores[0.2]['accuracy'], label='Train')
    
    
#fig.tight_layout()
plt.show()       


### Data Augmentation

In [ ]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                               shear_range=10,
                               zoom_range=0.1,
                               vertical_flip=True)

train_ds = train_gen.flow_from_directory('./clothing-dataset-small/train/',
                              target_size=(150,150),
                              batch_size=32)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = val_gen.flow_from_directory('./clothing-dataset-small/validation/',
                                     target_size=(150,150),
                                     batch_size=32,
                                     shuffle=False
                                    )

In [ ]:
lr = 0.001
size = 32
dr = 0.2

model = make_modelv3(learning_rate=lr, size_inner=size, droprate=dr)
history = model.fit(train_ds, epochs=30, validation_data=val_ds)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

ax = plt.plot(history.history['val_accuracy'], label='Val')
ax = plt.plot(history.history['accuracy'], label='Train')
    
plt.show()       


In [ ]:
history.history

In [ ]:
img2 = load_img('clothing-dataset-small/test/shoes/20209905-0830-4a6d-a1cf-7382051f8f18.jpg', target_size=(150,150))

x = np.array(img2)
X = np.array([x])
X.shape

X = preprocess_input(X)

pred = model.predict(X)

values = ['dress', 'hat', 'longsleeve',  'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']

list(zip(values, pred[0]))

In [ ]:
img2

#### Another way to do it - Practice

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras_preprocessing.image import load_img, ImageDataGenerator
from keras.applications.xception import Xception, preprocess_input, decode_predictions
from keras.applications.inception_v3 import InceptionV3,preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

In [5]:
base_model = tf.keras.applications.Xception(weights='imagenet',
                                            include_top=False,
                                            input_shape=(300, 300, 3))

#base_model = InceptionV3(include_top=False,
#                         input_shape=(150, 150, 3),
#                        weights = 'imagenet')
#
base_model.trainable = False

base_model.summary()

In [7]:
path ='./clothing-dataset-small/train/t-shirt/'
file = '1cab0eb2-60f6-4d0a-a380-9a07b834d855.jpg'
fullName = f'{path}/{file}'

#train_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                               #rotation_range=30,
                               #shear_range=0.2,
                               #zoom_range=0.2,
                               #horizontal_flip=True,
                               #width_shift_range=0.2,
                               #height_shift_range=0.2
#                              )
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                               
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True
                              )
#rescale=1/255.)

train_ds = train_gen.flow_from_directory('./clothing-dataset-small/train/',
                                         target_size=(300,300),                              
                                         batch_size=20)

#val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)#rescale=1/255.)

val_ds = val_gen.flow_from_directory('./clothing-dataset-small/validation/',
                                     target_size=(300,300),
                                     batch_size=20)

In [11]:
inputs = keras.Input(shape=(300,300,3))
base = base_model(inputs, training=False)

#last_layer_out = base_model.get_layer('mixed7')
#last_layer_out = base_model.get_layer('add_70')
#base = last_layer_out.output
#x = layers.Flatten()(base_model.output)
vectors = keras.layers.GlobalAveragePooling2D()(base)

#x = layers.Dense(16, activation='relu')(x)
inner = keras.layers.Dense(128, activation='relu')(vectors)
drop = keras.layers.Dropout(0.2)(inner)
outputs = keras.layers.Dense(10)(drop)
#model = keras.Model(inputs, outputs)

from tensorflow.keras import Model

#modelx = Model(base_model.input, x)
modelx = Model(inputs, outputs)

optimizer = keras.optimizers.Adam(learning_rate=0.001)
loss = keras.losses.CategoricalCrossentropy(from_logits=True)

modelx.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


In [13]:
history = modelx.fit(train_ds,
                     validation_data=val_ds,
                     steps_per_epoch=50,
                     epochs=15,
                     #validation_steps=5,
                     #verbose=2
                    )

Epoch 1/15
50/50 [==============================] - 23s 456ms/step - loss: 0.6901 - accuracy: 0.7650 - val_loss: 0.4976 - val_accuracy: 0.8328
Epoch 2/15
50/50 [==============================] - 21s 412ms/step - loss: 0.5296 - accuracy: 0.8067 - val_loss: 0.4266 - val_accuracy: 0.8475
Epoch 3/15
50/50 [==============================] - 20s 403ms/step - loss: 0.4784 - accuracy: 0.8400 - val_loss: 0.4036 - val_accuracy: 0.8563
Epoch 4/15
50/50 [==============================] - 21s 428ms/step - loss: 0.4341 - accuracy: 0.8550 - val_loss: 0.4176 - val_accuracy: 0.8534
Epoch 5/15
50/50 [==============================] - 20s 408ms/step - loss: 0.3851 - accuracy: 0.8660 - val_loss: 0.4229 - val_accuracy: 0.8563
Epoch 6/15
50/50 [==============================] - 20s 404ms/step - loss: 0.4069 - accuracy: 0.8530 - val_loss: 0.4079 - val_accuracy: 0.8710
Epoch 7/15
18/50 [=========>....................] - ETA: 12s - loss: 0.3357 - accuracy: 0.8806

KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

ax = plt.plot(history.history['val_accuracy'], label='Val')
ax = plt.plot(history.history['accuracy'], label='Train')
    
plt.show()    

In [ ]:
img2 = load_img('clothing-dataset-small/test/shoes/20209905-0830-4a6d-a1cf-7382051f8f18.jpg', target_size=(300,300))

x = np.array(img2)
X = np.array([x])
X = preprocess_input(X)

pred = modelx.predict(X)

values = ['dress', 'hat', 'longsleeve',  'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']

list(zip(values, pred[0]))

In [ ]:
img2

In [ ]:
img3 = load_img('clothing-dataset-small/test/shirt/4758e641-df13-4b3a-93a2-a9e232561c95.jpg', target_size=(300,300))

x = np.array(img2)
X = np.array([x])
X = preprocess_input(X)

pred = modelx.predict(X)

values = ['dress', 'hat', 'longsleeve',  'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']

list(zip(values, pred[0]))

In [ ]:
img3

In [ ]:
img2 = load_img('clothing-dataset-small/test/hat/31ae3916-c352-45fe-9866-a8947fc74991.jpg', target_size=(300,300))

x = np.array(img2)
X = np.array([x])
X = preprocess_input(X)

pred = modelx.predict(X)

values = ['dress', 'hat', 'longsleeve',  'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']

list(zip(values, pred[0]))

In [ ]:
img2

In [ ]:
from keras.preprocessing.image import img_to_array
from numpy import expand_dims
Image = img_to_array(img3)

New = expand_dims(Image, 0)
ImgGen = ImageDataGenerator(vertical_flip=True)


results = ImgGen.flow(New, batch_size=1)

batch = results.next()
image = batch[0].astype('uint8')
plt.imshow(image)
plt.show()